In [1]:

# class svpEngine:
#     def __init__(self, phrases_file, labels_file):
#         self.phrases = genfromtxt(phrases_file, delim=',')
#         self.labels = genfromtxt(labels_file, delim=',')

In [2]:
from nltk.tokenize import WordPunctTokenizer
from numpy import array, genfromtxt, where
from tensorflow.keras import Sequential, layers


In [7]:
embedding_size = 500
batch_size = 32

phrases_file='balance_words.csv'
labels_file='balance_labels.csv'

phrases=genfromtxt(phrases_file, comments=None, dtype=str, delimiter='|', skip_header=1, usecols=list(range(1, 25)))
labels = genfromtxt(labels_file, comments=None, dtype=str, delimiter='|', skip_header=1, usecols=list(range(1, 25)))

from sklearn.feature_extraction.text import CountVectorizer
import re

prep= lambda x: re.sub('\d', '_DIG_', x).lower()

cvec = CountVectorizer('(?u)\\b\\w\\w+\\b', preprocessor=prep)

from itertools import chain
cvec.fit(chain(*phrases))
num2word = { v:k for (k, v) in cvec.vocabulary_.items()}
word2num = cvec.vocabulary_
word2num[num2word[0]]=len(word2num)
word2num[num2word[1]]=len(word2num)
word2num[0]='0.0'
word2num[1]='O'
word2num = lambda word: cvec.vocabulary_[word] if word in cvec.vocabulary_ else 0.0 if word=='0.0' else 1.0

label_set = set(word for label in labels for word in label if (word and not word.isdigit()))
label_dict = {k:(v+2) for (k, v) in zip(list(label_set), range(len(list(label_set))))}
label_dict['0.0']=0
label_dict['O']=1
label2num = lambda word: label_dict[word]

X = array([ list(map(word2num, prep(phrase)) for phrase in phrases ])
y = array([ list(map(label2num, tokens)) for tokens in labels])

SyntaxError: invalid syntax (<ipython-input-7-905854f01722>, line 33)

In [21]:

import os
# dont use GPU
os.environ["CUDA_VISIBLE_DEVICES"]=""

toy = Sequential()
toy.add(layers.Embedding(24, embedding_size, input_length=len(X[0]), mask_zero=True))
toy.add(layers.LSTM(24, return_sequences=True))
toy.add(layers.Dropout(0.1))
toy.add(layers.Dense(24))
toy.add(layers.Activation('softmax'))
toy.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy())
toy.fit(X, y, batch_size=batch_size)

Train on 2690 samples
  32/2690 [..............................] - ETA: 7:07

InvalidArgumentError:  indices[22,1] = 671 is not in [0, 24)
	 [[node sequential_4/embedding_4/embedding_lookup (defined at C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_37031]

Function call stack:
distributed_function


In [ ]:
cvec.vocabulary_

In [11]:
y.shape

(2690, 24)

In [20]:
import tensorflow as tf
tf.losses.SparseCategoricalCrossentropy?

In [23]:
tf.losses.binary_crossentropy?